In [1]:
import txsim as tx
import scanpy as sc
import os
import pandas as pd
from anndata import AnnData
import numpy as np
#from scipy.spatial import ConvexHull, convex_hull_plot_2d
import xb.formatting as xf

In [2]:
maindir='../../data/unprocessed_adata'
files=os.listdir(maindir)

In [6]:
for fl in files:
    adata=sc.read(maindir+'/'+fl)
    adata=adata[:,adata.var['in_panel']]
    adata.layers['lognorm']=adata.X
    adata.layers['raw']=adata.X
    adata.uns['spots']['x']=adata.uns['spots']['x_location']
    adata.uns['spots']['y']=adata.uns['spots']['y_location']
    adata.obs['x']=list(adata.obs['x_centroid'])
    adata.obs['y']=list(adata.obs['y_centroid'])
    qlm1=tx.quality_metrics._combined.all_quality_metrics(adata)
    qlm1.columns=[fl[:-5]]
    qlm1.loc['area',fl[:-5]]=float(xf.cell_area(adata))
    qlm1.loc['reads_prop_qv>20',fl[:-5]]=np.sum(adata.uns['spots'].loc[:,'qv']>20)/adata.uns['spots'].shape[0]
    qlm1.loc['total_reads',fl[:-5]]=adata.uns['spots'].shape[0]
    qlm1.loc['prop_reads_in_panel',fl[:-5]]=np.sum(adata.uns['spots']['feature_name'].isin(adata.var.index[adata.var['in_panel']]))/adata.uns['spots'].shape[0]
    qlm1.loc['prop_reads_assigned_to_cells',fl[:-5]]=int(np.sum(adata.X))/np.sum(adata.uns['spots']['feature_name'].isin(adata.var.index[adata.var['in_panel']]))
    try: 
        out=pd.merge(out,qlm1,right_index=True,left_index=True)
    except:
        out=qlm1

In [7]:
out=out.round(2)

In [8]:
out.to_csv(r'../../figures/tables/supp1_raw_information.csv')